In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import math

import time
import pandas as pd
import folium as fl
import requests
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable, GeocoderQuotaExceeded
from geopy.extra.rate_limiter import RateLimiter
import re
quartier_url = 'https://data.montreal.ca/dataset/00bd85eb-23aa-4669-8f1b-ba9a000e3dd8/resource/e9b0f927-8f75-458c-8fda-b5da65cc8b73/download/limadmin.geojson'
Montreal = []

try:
    r2 = requests.get(quartier_url, timeout=30)

except requests.exceptions.RequestException as erreur2:
    print(f"Erreur de connexion à l'adresse web suivante {quartier_url} : {erreur2}")

else:
    if r2.status_code >= 200 and r2.status_code <= 299:
        Montreal = r2.json()
    else:
        print(f"Erreur de connexion à l'adresse web suivante {quartier_url} : erreur {r2.status_code}")


In [2]:
#Ameneties
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
#italian restaurants
italien = pd.read_csv('CSVs/Restaurant-Italien.csv')
new_italien = pd.read_csv('CSVs/new-towns-italien.csv')
#TODO  Rajouter le type de restaurant correspondant
italien[["Type"]] = "italien"
new_italien[["Type"]] = "italien"
Italy = pd.concat([italien,new_italien])
Italy = Italy.drop_duplicates()

#greek restaurants
Grec = pd.read_csv('CSVs/Restaurant-Grec.csv')
new_greek = pd.read_csv('CSVs/new-towns-greek.csv')
Grec[["Type"]] =  "grec"
new_greek[["Type"]] = "grec"
Greek = pd.concat([Grec,new_greek])
Greek = Greek.drop_duplicates()


#African Restaurants
Africain = pd.read_csv('CSVs/Restaurant Africain.csv')
Africain[["Type"]] = "africain"


#Antillais Restaurants
Antillais = pd.read_csv('CSVs/Restaurant-Antillais.csv')
Antillais[["Type"]] =  "antillais"

#Asian Restaurants
Asiatique = pd.read_csv('CSVs/Restaurant-Asiatique.csv')
new_asian = pd.read_csv("CSVs/new-towns-asian.csv")
Asiatique[["Type"]] =  "asiatique"
new_asian[["Type"]] = "asiatique"
Asian = pd.concat([Asiatique,new_asian])
Asian = Asian.drop_duplicates()

#Mexican Restaurants
Mexicain = pd.read_csv('CSVs/Restaurant-Mexicain.csv')
Mexicain[["Type"]] = "mexicain"

#Healthy Restaurants
Santé = pd.read_csv('CSVs/Restaurant-Santé.csv')
new_healthy = pd.read_csv("CSVs/new-towns-healthy.csv")
Santé[["Type"]] = "santé"
new_healthy[["Type"]] = "santé"
Healthy = pd.concat([Santé,new_healthy])
Healthy = Healthy.drop_duplicates()

#Coffee place
Café = pd.read_csv('CSVs/Café.csv')
new_coffee = pd.read_csv("CSVs/new-towns-coffee.csv")
Café[["Type"]] = "café"
new_coffee[["Type"]] = "café"
Coffee = pd.concat([Café,new_coffee])
Coffee = Coffee.drop_duplicates()
#All Restaurants



# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Neighbourhoods Data ( GPS coordinates + Area and Population)
neighborhoods = gpd.read_file("MTLTieks.geojson")
superficie = pd.read_csv('CSVs/Superficie_quartier.csv')
# --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Properties
Propriétés = pd.read_excel('CSVs/Centris_scraping2.xlsx')
## Clean the address list from Initial Property list
Addresse_list = Propriétés['Addresse'].tolist()
for i in range(len(Addresse_list)):
   ##TODO Retirer toute l'information qui vient après ", Montréal" (mettons (123 avenue xyz, Montréal, XYZ) --> (123, avenue xyz, Montréal))
   Addresse_list[i] = (Addresse_list[i].split(', Montréal')[0]+", Montréal") 

   ## Retirer toutes les lettres qui suiveront des numéro civique (Exemple 123A de la gauchetière --> 123 de la gauchetière)
   Addresse_list[i] = re.sub(r'\d+[A-Za-z]*', lambda x: x.group().rstrip('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdfghijklmnopqrstuvwxyz'), Addresse_list[i])   # address = Addresse_list[i]

   ## Ajouter un "e" lorsque nous avons des cas comme '20 rue --> 20e rue OU 8 avenue --> 8e avenue
   Addresse_list[i] = re.sub(r'^(\d+), (\d+) Avenue', r'\1, \2e Avenue', Addresse_list[i])
   Addresse_list[i] = re.sub(r'^(\d+), (\d+) Rue', r'\1, \2e Rue', Addresse_list[i])

   #TODO Nettoyer pour les appartements, les retirer exemple (123, app. 808, avneue xyz --> 123, avenue xyz)
   Addresse_list[i] = re.sub(r'app\..*?, ', '', Addresse_list[i])

   #TODO Retirer les Abbrévations de Pointes-aux-trembles et RDP des addresses
   Addresse_list[i] =Addresse_list[i].replace(" (R.-d.-P.)","")
   Addresse_list[i]=Addresse_list[i].replace(" (P.-a.-T.)","")

   #TODO Retirer les "quartier..." Exemple( 123, avenue xyz, Dorval, Quartier Sud --> 123, avenue xyz, Dorval)
   Addresse_list[i] = re.sub(", Quartier .*, ", ", ",Addresse_list[i])

   #TODO Retirer les addresses double (123-125 avenue xyz --> 123, avenue xyz)
   Addresse_list[i] = re.sub(r' - \d+', '', Addresse_list[i])

   #TODO Nettoyer pour les appartements, les retirer exemple (123, app. 808, avneue xyz --> 123, avenue xyz)
   Addresse_list[i] = (re.sub(r'app\.\s*([A-Za-z\d]+, )', '', Addresse_list[i]))
   match = re.search(r"app\.", Addresse_list[i])
   if match:
      Addresse_list[i] = re.sub(r"app\.\s\d+, ", "", Addresse_list[i])

   #TODO enlever les d'
   Addresse_list[i] = Addresse_list[i].replace(" d'"," ")
Propriétés['Addresse'] = Addresse_list



In [3]:
## Define the function to extract different interest place around
def get_place_data(url,df):
    driver = webdriver.Chrome()
    driver.get(f"{url}{1}")

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    container = soup.find_all("div", {"class":"col-auto"})
    for i in container:
        if i.find("h2") is not None:
            result = i.find("h2")
    #result= [i.find("h2") for i in container if i.find("h2") is not None]
    stringresult = (result.text).split("sur ")
    intresult = int((stringresult[1]).split(" ")[0])
    iterations = ((intresult//100)+2)
    quartier = []
    centers = []
    location = []
    for i in range(1,iterations):
        time.sleep(5)
        driver.get(f"{url}{i}")
        # Récupérez le contenu HTML de la page
        html = driver.page_source
        # Utilisez BeautifulSoup pour parser le contenu HTML
        soup = BeautifulSoup(html, "html.parser")
        all_centers = (soup.find_all("div", {"class": "list-group-item-title"}))
        centers.extend([center.text for center in all_centers])
        address_content = (soup.find_all("div", {"class": "list-group-item-content"}))
        for address in address_content:
            if len(address) == 1:
                city = address.find_next("div")
            elif len(address) == 2:
                loc = address.find_next("div")
                city = loc.find_next("div")
            elif len(address) == 3:
                badge = address.find_next("div")
                loc = badge.find_next("div")
                city = loc.find_next("div")
            quartier.append(city.text)
            location.append(loc.text)
    df = pd.DataFrame({"Centre": centers, "Addresse": location, "Quartier": quartier})
    return df

##TODO Transforme l'adresse en Point géographique (Lat,Lon,Alt) 123 avenue xyz, montréal --> (-73,45,0)
def get_location(df):
    locator = Nominatim(user_agent="myGeocoder", timeout=10)
    df['location'] = df['Addresse'].apply(locator.geocode)
    df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)


    return df
#TODO Découper le point géogrpahique en 3 colonnes séparés
def get_lat_lon(df):
    df[['Lat', 'Lon', 'Alt']] = df['point'].apply(pd.Series)
    df['Lat'] = df['Lat'].astype(float)
    df['Lon'] = df['Lon'].astype(float)
    df = df.drop(columns=['point', 'Alt'])
    return df

#TODO Aller chercher les limites geJSOM d'un quartier et trouver dans quel qaurtier appartient "x" point géogrpahique Lon,Lat Exemple -73.5,45.22 --> Verdun
def Link_map_data(df):
    df["geometry"] = gpd.points_from_xy(df["Lon"], df["Lat"])
    df = gpd.GeoDataFrame(df, geometry="geometry")
    df1 = gpd.sjoin(df, neighborhoods, how="left", op='within')
# Ajoutez une colonne de quartier à votre dataframe de df
    df["NOM"] = df1["NOM"]
    if 'location' in df.columns:
        df =df.drop(columns=['location'])
    if 'geometry' in df.columns:
        df = df.drop(columns=['geometry'])

    return df

def apply_data_map(dforiginal,dfgrouped,dffinal):
    dffinal = pd.merge(dforiginal,dfgrouped, on ="NOM", how="inner")
    return

Prop_map = get_location(Propriétés)
Prop_map = get_lat_lon(Prop_map)
Prop_map = Link_map_data(Prop_map)
Prop_map.to_csv("Export/Houses2V1.csv")

In [ ]:
## Call the function on interest places
Sportscenters = pd.DataFrame
sportsurl = "https://montreal.ca/lieux?filter=true&mtl_content.lieux.category.code=CENT&page="
Sportscenters = get_place_data(sportsurl,Sportscenters)
Sportscenters["Addresse"] = Sportscenters["Addresse"].apply(lambda x: x + ", Montréal")
Sportscenters = get_location(Sportscenters)
Sportscenters=get_lat_lon(Sportscenters)
Sportscenters = Link_map_data(Sportscenters)



Communitycenters = pd.DataFrame
communityurl = "https://montreal.ca/lieux?mtl_content.lieux.category.code=BIBL&page="
Communitycenters = get_place_data(communityurl,Communitycenters)
Communitycenters["Addresse"] = Communitycenters["Addresse"].apply(lambda x: x + ", Montréal")
Communitycenters = get_location(Communitycenters)
Communitycenters=get_lat_lon(Communitycenters)
Communitycenters = Link_map_data(Communitycenters)


Parcs = pd.DataFrame
parcsurl = "https://montreal.ca/lieux?filter=true&mtl_content.lieux.category.code=PARC&page="
Parcs = get_place_data(parcsurl,Parcs)
Parcs["Addresse"] = Parcs["Addresse"].apply(lambda x: x + ", Montréal")
Parcs = get_location(Parcs)
Parcs=get_lat_lon(Parcs)
Parcs = Link_map_data(Parcs)



Mairies = pd.DataFrame()
mairiesurl = "https://montreal.ca/lieux?mtl_content.lieux.installation.code=MAIR&PAGE="
Mairies = get_place_data(mairiesurl,Mairies)
Mairies = Mairies[Mairies['Centre'].str.contains("Mairie d'arrondissement")].reset_index().drop(columns=["index"])
Mairies['Addresse'].replace(["5160, boul. Décarie, bureau 600","201 de l'avenue Laurier Est"], ["5160, boul. Décarie, Montreal", "201, avenue Laurier Est, Montreal"], inplace=True)
Mairies = get_location(Mairies)
Mairies = get_lat_lon(Mairies)
Mairies = Link_map_data(Mairies)



In [11]:
Parcs.to_csv("Export/Parcs.csv")
Communitycenters.to_csv("Export/Communitycenters.csv")
Sportscenters.to_csv("Export/Sportscenters.csv")
Mairies.to_csv("Export/Mairies.csv")

In [ ]:
new_restaurants = pd.concat([Coffee,Greek,Italy,Africain,Antillais,Asian,Mexicain,Healthy])
#TODO Drop duplicates si toutes les colonnes sauf la catégogire de l'origine du restaurant est pareille.
new_restaurants = new_restaurants.drop_duplicates(subset=["Nom","Rue","postalCode","Latitude","Longitude","reviewsCount","totalScore"])
new_restaurants = new_restaurants.rename(columns={'Longitude':'Lon','Latitude':'Lat'})
Link_map_data(new_restaurants)
nbNew = new_restaurants.groupby("NOM").count()[["Nom"]]
nbNew = nbNew.rename(columns={"Nom":"Count of Restaurant"})
new_restaurants = new_restaurants.dropna(subset=["NOM"])
new_restaurants = new_restaurants.drop(columns=["geometry"])
new_restaurants.to_csv("Export/Restaurants.csv")


In [ ]:
#TODO Drop data on houses where either name, number of rooms or price is NaN
price_per_piece = Prop_map.dropna(subset=["piece","NOM","price"], inplace=False)
#TODO 123 456,54$ type String --> 123456 type INT
clean_price = [int((re.sub(r'\u00A0', ' ', r[2])).replace(" ","").replace("$","")) for i,r in price_per_piece.iterrows()]
price_per_piece["price"] = clean_price
#TODO retirer pièce du nombre de pièces --> 13 pièces --> 13
clean_piece = [int(r[3].replace(" pièce","").replace("s","")) for i,r in price_per_piece.iterrows()]
price_per_piece['piece'] = clean_piece
#TODO Prix/pièce
price_per_piece["Price_per_piece"] = (price_per_piece["price"]/price_per_piece["piece"]).astype(int)

price_per_piece = price_per_piece.groupby("NOM").mean("Price_per_piece")[["Price_per_piece"]].astype(int).sort_values(by="Price_per_piece",ascending = False).reset_index()
price_per_piece.to_csv("Export/Price_per_piece.csv")

In [5]:
rest_per_km = pd.merge(superficie[["NOM","Superficie (km2)"]],nbNew,on="NOM")
rest_per_km["Restaurant_by_km2"] = (rest_per_km["Count of Restaurant"]/rest_per_km["Superficie (km2)"]).apply(math.ceil)
rest_per_km.to_csv("Export/Rest_per_KM.csv")